In [57]:
import pandas as pd
COLUMN_NAMES =["PLAYER", "COLLEGE", "CONF", "GAMES_PLAYED", "GAMES_STARTED", "RECORD", "CMP", "ATT", "YDS", "TD", "INT", "RUSH_ATT", "RUSH_YDS", "RUSH_TD", "COLL_CMP", "COLL_ATT", "COLL_YDS", "COLL_TD", "COLL_INT", "COLL_RUSH_ATT", "COLL_RUSH_YDS", "COLL_RUSH_TDS"]
df = pd.read_csv('data/scraped_data.csv', usecols=COLUMN_NAMES)

In [58]:
REGRESSOR_COLUMN_NAMES = ["GAMES_PLAYED", "GAMES_STARTED", "CMP", "ATT", "YDS", "TD", "INT", "RUSH_ATT", "RUSH_YDS", "RUSH_TD"]
CLASS_LABELS = df['RECORD'].apply(lambda rec: 0 if int(rec.split('-')[0]) < 10 else 1)
REGRESSOR_LABELS = df[REGRESSOR_COLUMN_NAMES]
TRAIN_DATA = df.drop(columns=REGRESSOR_COLUMN_NAMES)
TRAIN_DATA.drop(columns=['PLAYER', 'RECORD'], inplace=True)

In [59]:
#one-hot encoding
# first, combine the 2 dataframes
TEST_COLUMN_NAMES =["PLAYER", "COLLEGE", "CONF", "COLL_CMP", "COLL_ATT", "COLL_YDS", "COLL_TD", "COLL_INT", "COLL_RUSH_ATT", "COLL_RUSH_YDS", "COLL_RUSH_TDS"]
test_df = pd.read_csv('data/test_data.csv', usecols=TEST_COLUMN_NAMES)
combined_df = TRAIN_DATA.append(test_df.drop(columns=["PLAYER"]), ignore_index=True)

In [60]:
# one-hot encoding
one_hot = pd.get_dummies(combined_df)

ENCODED_TRAIN_DATA = one_hot.iloc[:225, :]
ENCODED_TEST_DATA = one_hot.iloc[225:, :]
college_test_df = test_df.merge(ENCODED_TEST_DATA)
college_test_df.drop(columns=['COLLEGE', 'CONF'], inplace=True)
college_test_df

,PLAYER,COLL_CMP,COLL_ATT,COLL_YDS,COLL_TD,COLL_INT,COLL_RUSH_ATT,COLL_RUSH_YDS,COLL_RUSH_TDS,COLLEGE_Akron,...,CONF_CUSA,CONF_Ind,CONF_MAC,CONF_MVC,CONF_MWC,CONF_Pac-10,CONF_Pac-12,CONF_SEC,CONF_Sun Belt,CONF_WAC
0,Kyler Murray,331,482,5098,48,14,190,1369,12,0,...,0,0,0,0,0,0,0,0,0,0
1,Will Grier,621,945,8556,81,23,147,148,7,0,...,0,0,0,0,0,0,0,0,0,0
2,Dwayne Haskins,388,553,5145,51,9,97,208,4,0,...,0,0,0,0,0,0,0,0,0,0
3,Ryan Finley,919,1429,11008,62,28,209,386,5,0,...,0,0,0,0,0,0,0,0,0,0
4,Shea Patterson,426,682,5503,44,17,154,421,3,0,...,0,0,0,0,0,0,0,0,0,0
5,Justin Herbert,522,832,6904,62,17,169,517,9,0,...,0,0,0,0,0,0,1,0,0,0
6,Drew Lock,860,1515,11820,96,39,196,407,9,0,...,0,0,0,0,0,0,0,1,0,0
7,Kyle Shurmur,704,1227,8579,63,29,133,-340,3,0,...,0,0,0,0,0,0,0,1,0,0
8,Gardner Minshew,726,1119,7964,60,20,90,27,3,0,...,0,0,0,0,0,0,1,0,0,0
9,Nathan Stanley,415,726,5132,49,15,81,-93,1,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# test some classifiers
X = ENCODED_TRAIN_DATA
y = CLASS_LABELS
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [71]:
#rfc
from sklearn.ensemble import RandomForestClassifier

In [75]:
rf_model = RandomForestClassifier(n_estimators=1, random_state=42)
rf_model.fit(X_train, y_train)
test_score = rf_model.score(X_test, y_test)
test_score

0.5789473684210527

In [91]:
rf_model.predict(ENCODED_TEST_DATA)

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1])

In [76]:
# dt
from sklearn.tree import DecisionTreeClassifier

In [89]:
dt_model = DecisionTreeClassifier(max_depth=8, random_state=42)
dt_model.fit(X_train, y_train)
test_score = dt_model.score(X_test, y_test)
test_score

0.5614035087719298

In [90]:
dt_model.predict(ENCODED_TEST_DATA)

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [92]:
# MLP
from sklearn.neural_network import MLPClassifier

In [121]:
mlp_model = MLPClassifier(random_state=42, hidden_layer_sizes=(8, 8), max_iter=4000)
mlp_model.fit(X_train, y_train)
test_score = mlp_model.score(X_test, y_test)
test_score

0.5614035087719298

In [120]:
mlp_model.predict(ENCODED_TEST_DATA)

array([0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0])

In [122]:
# regressors
X = ENCODED_TRAIN_DATA
y = REGRESSOR_LABELS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [123]:
from sklearn.ensemble import RandomForestRegressor

In [137]:
regr = RandomForestRegressor(max_depth=1, random_state=42, n_estimators=100)
regr.fit(X_train, y_train)
test_score = regr.score(X_test, y_test)
test_score

-0.04056840109189851

In [138]:
predictions = regr.predict(ENCODED_TEST_DATA)
list(predictions[9])

[35.3262472562038,
 28.856862676712993,
 588.3158943654003,
 966.8500451448854,
 6764.525918232187,
 40.78911176144185,
 26.98878432396704,
 91.45231695773282,
 377.23895414018165,
 3.4561936759994243]